In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

# Functions

In [2]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [3]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,method
0,0.097160,exp1,Sphere,1.0,NN_HMu
1,0.097216,exp1,Sphere,1.0,NN_HMu
2,0.096804,exp1,Sphere,1.0,NN_HMu
3,0.097892,exp1,Sphere,1.0,NN_HMu
4,0.097108,exp1,Sphere,1.0,NN_HMu


In [4]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])

In [5]:
print(table2.to_csv(float_format='%.2f'))

experiment,,exp1,exp1,exp1,exp2,exp2,exp2,exp3,exp3,exp3,exp4,exp4,exp4
freq,,0.5,1.0,4.0,0.5,1.0,4.0,0.5,1.0,4.0,0.5,1.0,4.0
function,method,,,,,,,,,,,,
Rastrigin,NN_CwN,0.19,0.10,0.02,0.20,0.10,0.02,0.20,0.10,0.02,0.19,0.09,0.02
Rastrigin,NN_HMu,0.20,0.10,0.03,0.19,0.10,0.02,0.20,0.10,0.02,0.19,0.10,0.02
Rastrigin,NN_No,0.20,0.10,0.02,0.20,0.10,0.02,0.20,0.10,0.02,0.20,0.09,0.02
Rastrigin,NN_RI,0.21,0.11,0.03,0.21,0.11,0.03,0.21,0.11,0.03,0.20,0.10,0.02
Rosenbrock,NN_CwN,0.20,0.10,0.02,0.19,0.10,0.02,0.20,0.10,0.02,0.19,0.09,0.02
Rosenbrock,NN_HMu,0.19,0.10,0.02,0.19,0.10,0.02,0.20,0.10,0.02,0.19,0.10,0.02
Rosenbrock,NN_No,0.20,0.10,0.02,0.19,0.10,0.02,0.20,0.10,0.02,0.19,0.09,0.02
Rosenbrock,NN_RI,0.20,0.10,0.03,0.22,0.14,0.02,0.20,0.10,0.03,0.19,0.10,0.02
Sphere,NN_CwN,0.19,0.10,0.02,0.20,0.10,0.02,0.27,0.10,0.02,0.19,0.10,0.02
Sphere,NN_HMu,0.19,0.10,0.02,0.20,0.10,0.02,0.20,0.10,0.02,0.19,0.10,0.02
Sphere,NN_No,0.20,0.10,0.02,0.19,0.10,0.03,0.20,0.10,0.02,0.19,0.10,0.02
Sphere,NN

In [6]:
table2

experiment             exp1                          exp2                      \
freq                    0.5       1.0       4.0       0.5       1.0       4.0   
function   method                                                               
Rastrigin  NN_CwN  0.194497  0.097338  0.024353  0.196570  0.099296  0.024606   
           NN_HMu  0.201269  0.099626  0.025077  0.194138  0.098442  0.024768   
           NN_No   0.197377  0.099105  0.024398  0.195575  0.098174  0.024504   
           NN_RI   0.207512  0.109726  0.025569  0.214207  0.107151  0.029630   
Rosenbrock NN_CwN  0.195657  0.097735  0.024212  0.191858  0.097053  0.024276   
           NN_HMu  0.194581  0.096706  0.024811  0.193950  0.098986  0.024603   
           NN_No   0.197014  0.096260  0.024484  0.194246  0.099463  0.024367   
           NN_RI   0.200048  0.104496  0.033173  0.221169  0.140467  0.023473   
Sphere     NN_CwN  0.190852  0.097568  0.024553  0.195001  0.096490  0.024671   
           NN_HMu  0.193010  0.097502  0.024693  0.196681  0.098865  0.024475   
           NN_No   0.199657  0.096912  0.024482  0.193673  0.098057  0.033524   
           NN_RI   0.200972  0.106642  0.024639  0.211199  0.101087  0.029282   

experiment             exp3                          exp4                      
freq                    0.5       1.0       4.0       0.5       1.0       4.0  
function   method                                                              
Rastrigin  NN_CwN  0.199848  0.098637  0.024900  0.189137  0.093953  0.023462  
           NN_HMu  0.197705  0.099152  0.024971  0.191700  0.098307  0.024105  
           NN_No   0.200952  0.100575  0.024392  0.195706  0.094887  0.023929  
           NN_RI   0.206613  0.105577  0.025767  0.196920  0.097911  0.024217  
Rosenbrock NN_CwN  0.200414  0.100184  0.024810  0.192678  0.094701  0.023370  
           NN_HMu  0.199590  0.101151  0.024951  0.193674  0.097333  0.024650  
           NN_No   0.201891  0.100589  0.024773  0.192888  0.094568  0.024335  
           NN_RI   0.202411  0.103278  0.025373  0.191380  0.096475  0.024741  
Sphere     NN_CwN  0.271572  0.099958  0.024798  0.187921  0.095333  0.023903  
           NN_HMu  0.199215  0.099362  0.024720  0.191051  0.096827  0.024767  
           NN_No   0.201444  0.100451  0.024957  0.194790  0.097586  0.024481  
           NN_RI   0.204199  0.102339  0.025587  0.195909  0.097366  0.025428